In [52]:
import numpy as np
import  pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV , learning_curve

In [53]:
df = pd.read_csv(r"C:\Users\Rag9704\Desktop\financecode\Book4.csv")
print(df.head())
df= df[['X1','X2','X3','X4','X5','Label']]
df['X1'] = df['X1'].convert_objects(convert_numeric = True)
df=df.dropna()
X = df.drop('Label',axis=1)
y = df['Label']
df.info()


   0         X1     X2      X3     X4     X5  Label
0  1  -4.31E-01  0.868 -0.9260 -0.927 -0.569    0.0
1  2   3.39E-01  0.562  0.9310 -0.916 -0.545    0.0
2  3  -4.29E-01  0.871 -0.9280 -0.775 -0.583    0.0
3  4  -4.33E-01  0.861 -0.9220 -0.880 -0.500    0.0
4  5  -9.99E-01 -0.999  0.0775 -0.842 -0.515    0.0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 199 entries, 0 to 199
Data columns (total 6 columns):
X1       199 non-null float64
X2       199 non-null float64
X3       199 non-null float64
X4       199 non-null float64
X5       199 non-null float64
Label    199 non-null float64
dtypes: float64(6)
memory usage: 10.9 KB


c:\users\rag9704\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  after removing the cwd from sys.path.


In [54]:
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.5,random_state=0)

In [69]:
clf_entropy_no_ada = DecisionTreeClassifier(criterion = "entropy", random_state = 100,
                                            max_depth = 10,min_samples_leaf=5)
clf_entropy_no_ada.fit(X_train,y_train)
y_predict_no_ada = clf_entropy_no_ada.predict(X_test)
print("accuracy is :",accuracy_score(y_test,y_predict_no_ada)*100)

#decision tree with boosting
clf_entropy_ada = AdaBoostClassifier(base_estimator = clf_entropy_no_ada ,n_estimators=50,
                                     learning_rate =0.5)
clf_entropy_ada.fit(X_train,y_train)
y_predict_ada = clf_entropy_ada.predict(X_test)
print("accuracy is :",accuracy_score(y_test,y_predict_ada)*100) 

accuracy is : 70.0
accuracy is : 70.0


In [71]:
param_grid = {"base_estimator__criterion" : ["gini","entropy"],"base_estimator__splitter":["best", "random"],
             "n_estimators" : [50,100],"learning_rate" :[0.5,2]}

DTC = DecisionTreeClassifier(criterion = "entropy", random_state = 100,
                                            max_depth = 10,min_samples_leaf=5)
ABC = AdaBoostClassifier(base_estimator = DTC)
grid_search_ABC = GridSearchCV(ABC, param_grid = param_grid, scoring = 'accuracy')
grid_result = grid_search_ABC.fit(X_train,y_train)

print("Best : %f using %s" % (grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean,stddev,param in zip(means,stds,params):
    print("%f(%f) with: %r" % (mean,stddev,param))

c:\users\rag9704\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best : 0.777778 using {'base_estimator__criterion': 'gini', 'base_estimator__splitter': 'best', 'learning_rate': 2, 'n_estimators': 100}
0.737374(0.041977) with: {'base_estimator__criterion': 'gini', 'base_estimator__splitter': 'best', 'learning_rate': 0.5, 'n_estimators': 50}
0.737374(0.029193) with: {'base_estimator__criterion': 'gini', 'base_estimator__splitter': 'best', 'learning_rate': 0.5, 'n_estimators': 100}
0.757576(0.060201) with: {'base_estimator__criterion': 'gini', 'base_estimator__splitter': 'best', 'learning_rate': 2, 'n_estimators': 50}
0.777778(0.048122) with: {'base_estimator__criterion': 'gini', 'base_estimator__splitter': 'best', 'learning_rate': 2, 'n_estimators': 100}
0.717172(0.015800) with: {'base_estimator__criterion': 'gini', 'base_estimator__splitter': 'random', 'learning_rate': 0.5, 'n_estimators': 50}
0.707071(0.008865) with: {'base_estimator__criterion': 'gini', 'base_estimator__splitter': 'random', 'learning_rate': 0.5, 'n_estimators': 100}
0.717172(0.056

c:\users\rag9704\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [62]:
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train,y_train)
y_pred = svclassifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[39 10]
 [18 33]]
              precision    recall  f1-score   support

         0.0       0.68      0.80      0.74        49
         1.0       0.77      0.65      0.70        51

    accuracy                           0.72       100
   macro avg       0.73      0.72      0.72       100
weighted avg       0.73      0.72      0.72       100



In [46]:
svclassifier = SVC(kernel='poly',degree = 12,gamma = 'scale')
svclassifier.fit(X_train,y_train)
y_pred = svclassifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[36 13]
 [12 39]]
              precision    recall  f1-score   support

         0.0       0.75      0.73      0.74        49
         1.0       0.75      0.76      0.76        51

    accuracy                           0.75       100
   macro avg       0.75      0.75      0.75       100
weighted avg       0.75      0.75      0.75       100



In [33]:
svclassifier = SVC(kernel='rbf',gamma = 'scale')
svclassifier.fit(X_train,y_train)
y_pred = svclassifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[39 10]
 [18 33]]
              precision    recall  f1-score   support

         0.0       0.68      0.80      0.74        49
         1.0       0.77      0.65      0.70        51

    accuracy                           0.72       100
   macro avg       0.73      0.72      0.72       100
weighted avg       0.73      0.72      0.72       100



In [79]:
svclassifier = SVC(kernel='sigmoid',gamma = 'scale')
svclassifier.fit(X_train,y_train)
y_pred = svclassifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[39 10]
 [18 33]]
              precision    recall  f1-score   support

         0.0       0.68      0.80      0.74        49
         1.0       0.77      0.65      0.70        51

    accuracy                           0.72       100
   macro avg       0.73      0.72      0.72       100
weighted avg       0.73      0.72      0.72       100

